In [2]:
import cv2
import numpy as np

# Load the stereo images
left_img = cv2.imread('im1.jpeg', cv2.IMREAD_GRAYSCALE)
right_img = cv2.imread('im2.jpeg', cv2.IMREAD_GRAYSCALE)

# Define the number of inner corners of the chessboard (for example, an 8x8 chessboard has 7x7 inner corners)
pattern_size = (7, 7)

# Find the chessboard corners in both images
ret_left, corners_left = cv2.findChessboardCorners(left_img, pattern_size)
ret_right, corners_right = cv2.findChessboardCorners(right_img, pattern_size)

if ret_left and ret_right:
    # Refine the detected corners
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    cv2.cornerSubPix(left_img, corners_left, (11, 11), (-1, -1), criteria)
    cv2.cornerSubPix(right_img, corners_right, (11, 11), (-1, -1), criteria)

    # These are your 2D points in each image
    points_2d_left = corners_left
    points_2d_right = corners_right
else:
    print("Could not find chessboard corners in one or both images.")


Could not find chessboard corners in one or both images.


In [3]:
left_img = cv2.imread('im1.jpeg', cv2.IMREAD_GRAYSCALE)
right_img = cv2.imread('im2.jpeg', cv2.IMREAD_GRAYSCALE)

In [4]:
cv2.findChessboardCorners(left_img, (7,5))

(False, None)

In [ ]:
import cv2
import numpy as np

# 1. Load Images
img1 = cv2.imread("first_image.jpg")
img2 = cv2.imread("second_image.jpg")

# 2. Given 3D points and corresponding 2D points
# Replace with your data
object_points_left = np.array([
    [0, 1.5, 3.8],
    [0, 22.5, 3.8],
    [0, 22.5, 18.8],
    [0, 1.5, 18.8],
])

object_points_right = np.array([
    [24.3, 0, 3.8],
    [0, 22.5, 3.8],
    [0, 22.5, 18.8],
    [0, 1.5, 18.8],
])

# 3. Get 2D points (e.g., clicked on the images or detected automatically)
# For demonstration, let's assume they are:
image_points_left = np.array([
    # Example points, replace with detected coordinates
    [100, 100],
    [200, 100],
    [200, 200],
    [100, 200],
    [100, 50],
    [200, 50],
    [200, 250],
    [100, 250]
])

image_points_right = image_points_left + 5  # Just for demo, consider it's shifted by 5 pixels

# 4. Camera Calibration
_, camera_matrix_left, dist_coeff_left, _, _ = cv2.calibrateCamera([object_points], [image_points_left], img1.shape[:2], None, None)
_, camera_matrix_right, dist_coeff_right, _, _ = cv2.calibrateCamera([object_points], [image_points_right], img2.shape[:2], None, None)

# 5. Calculate Fundamental Matrix
F, _ = cv2.findFundamentalMat(image_points_left, image_points_right, cv2.FM_LMEDS)

# 6. Triangulation to get 3D coordinates
def triangulate_points(p1, p2, m1, m2):
    return cv2.triangulatePoints(m1, m2, p1.T, p2.T).T

# 7. Mouse Callback Function
def callback(event, x, y, flags, params):
    if event == cv2.EVENT_LBUTTONDOWN:
        p1 = np.array([x, y, 1]).reshape(3, 1)
        
        # Draw a + sign at p
        cv2.drawMarker(img1, (x, y), (0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=10)
        
        # Compute epipolar line on right image
        l2 = np.dot(F, p1)
        x0, y0 = map(int, [0, -l2[2]/l2[1]])
        x1, y1 = map(int, [img2.shape[1], -(l2[2] + l2[0]*img2.shape[1]) / l2[1]])
        
        # Draw the line on the right image
        cv2.line(img2, (x0, y0), (x1, y1), (0, 255, 0), 1)
        
        # This is a naive way to find a corresponding point. In real-world applications, you'd use better matching techniques.
        p2 = np.array([x+5, y, 1]).reshape(3, 1)  # Assuming the shift is 5 pixels for demonstration
        
        # Draw a + sign at p'
        cv2.drawMarker(img2, (x+5, y), (0, 0, 255), markerType=cv2.MARKER_CROSS, markerSize=10)
        
        # Get 3D coordinates
        point_3d_homo = triangulate_points(p1.T, p2.T, camera_matrix_left, camera_matrix_right)
        point_3d = (point_3d_homo / point_3d_homo[3])[:3].ravel()
        
        print(f"3D Point: {point_3d}")
        
        cv2.imshow('Left Image', img1)
        cv2.imshow('Right Image', img2)

# Create window and bind function to window
cv2.namedWindow('Left Image')
cv2.setMouseCallback('Left Image', callback)

cv2.imshow('Left Image', img1)
cv2.imshow('Right Image', img2)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [1]:
import cv2

# Global variables
point_list = []
point_count = 0

# Mouse callback function
def select_points(event, x, y, flags, param):
    global point_list, point_count
    if event == cv2.EVENT_LBUTTONDOWN:
        cv2.circle(temp_img, (x, y), 5, (0, 0, 255), -1)
        cv2.imshow('Image', temp_img)
        point_list.append((x, y))
        point_count += 1

        if point_count == 18:
            cv2.destroyAllWindows()

# Read your image
img_left = cv2.imread('im1.jpeg')
img_right = cv2.imread('im2.jpeg')

# For left image
temp_img = img_left.copy()
cv2.imshow('Image', temp_img)
cv2.setMouseCallback('Image', select_points)
cv2.waitKey(0)

points_2d_left = point_list.copy()  # save the points for the left image
point_list.clear()  # clear the list to store points for the right image
point_count = 0  # reset point count

# For right image
temp_img = img_right.copy()
cv2.imshow('Image', temp_img)
cv2.setMouseCallback('Image', select_points)
cv2.waitKey(0)

points_2d_right = point_list

print("Points in left image:", points_2d_left)
print("Points in right image:", points_2d_right)


Points in left image: [(253, 378), (85, 351), (92, 200), (260, 192), (407, 356), (294, 378), (301, 196), (413, 220)]
Points in right image: [(257, 404), (106, 365), (113, 197), (269, 177), (475, 376), (312, 402), (321, 184), (480, 214)]


In [10]:
import cv2
import numpy as np

# Given data
world_points_left = np.array([
    [  0.        ,  14.46666667,   3.8       ],
       [  0.        , 217.        ,   3.8       ],
       [  0.        , 217.        ,  18.8       ],
       [  0.        ,  14.46666667,  18.8       ],
], dtype=np.float32)

world_points_right = np.array([
 [394.        ,   0.        ,   3.8       ],
       [ 53.50617284,   0.        ,   3.8       ],
       [ 53.50617284,   0.        ,  18.8       ],
       [394.        ,   0.        ,  18.8       ],
], dtype=np.float32)

# Image dimensions
width = 1280
height = 720

# Approximate intrinsic matrix
fx = width
fy = height
cx = width / 2
cy = height / 2
intrinsic_matrix = np.array([[fx, 0, cx],
                             [0, fy, cy],
                             [0, 0, 1]], dtype=np.float32)


points_2d_left = np.array([(251, 379), (84, 352), (91, 201), (259, 190)], dtype=np.float32)
points_2d_right = np.array([(475, 373), (312, 402), (323, 185), (478, 214)], dtype=np.float32)

# Camera Calibration
ret, mtx_left, dist_left, rvecs_left, tvecs_left = cv2.calibrateCamera([world_points_left], [points_2d_left], (1280, 720), intrinsic_matrix, None)
ret, mtx_right, dist_right, rvecs_right, tvecs_right = cv2.calibrateCamera([world_points_right], [points_2d_right], (1280, 720), intrinsic_matrix, None)

# Calculate Fundamental Matrix
F, _ = cv2.findFundamentalMat(points_2d_left, points_2d_right, cv2.FM_8POINT)

print("Fundamental Matrix F:\n", F)

# Callback function for mouse click on the left image
def on_click_left(event, x, y, flags, param):
    global points_2d_left, points_2d_right, mtx_left, mtx_right, F

    if event == cv2.EVENT_LBUTTONDOWN:
        pt = np.array([x, y, 1]).reshape(3, 1)
        epiline = np.dot(F, pt).ravel()

        # Draw the epipolar line on the right image
        cv2.line(right_img, (0, int(-epiline[2] / epiline[1])), (right_img.shape[1], int(-(epiline[2] + epiline[0] * right_img.shape[1]) / epiline[1])), (0, 0, 255), 1)

        # Draw "+" on the left image
        cv2.drawMarker(left_img, (x, y), (0, 255, 0), markerType=cv2.MARKER_CROSS, markerSize=10, thickness=2)

        # Show the images again
        cv2.imshow('Left Image', left_img)
        cv2.imshow('Right Image', right_img)

# Callback function for mouse click on the right image
def on_click_right(event, x, y, flags, param):
    global points_2d_left, points_2d_right, mtx_left, mtx_right, F

    if event == cv2.EVENT_LBUTTONDOWN:
        pt = np.array([x, y, 1]).reshape(3, 1)
        epiline = np.dot(F.T, pt).ravel()

        # Draw the epipolar line on the left image
        cv2.line(left_img, (0, int(-epiline[2] / epiline[1])), (left_img.shape[1], int(-(epiline[2] + epiline[0] * left_img.shape[1]) / epiline[1])), (0, 0, 255), 1)

        # Draw "+" on the right image
        cv2.drawMarker(right_img, (x, y), (0, 255, 0), markerType=cv2.MARKER_CROSS, markerSize=10, thickness=2)

        # Show the images again
        cv2.imshow('Left Image', left_img)
        cv2.imshow('Right Image', right_img)

# Load your stereo images
left_img = cv2.imread('im1.jpeg')
right_img = cv2.imread('im2.jpeg')

cv2.namedWindow('Left Image')
cv2.setMouseCallback('Left Image', on_click_left)
cv2.namedWindow('Right Image')
cv2.setMouseCallback('Right Image', on_click_right)

cv2.imshow('Left Image', left_img)
cv2.imshow('Right Image', right_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


error: OpenCV(4.7.0) /Users/xperience/GHA-OCV-Python/_work/opencv-python/opencv-python/opencv/modules/calib3d/src/calibration.cpp:1576: error: (-5:Bad argument) For non-planar calibration rigs the initial intrinsic matrix must be specified in function 'cvCalibrateCamera2Internal'


In [3]:
import numpy as np

# Define your 3D and 2D points
points_3d = np.array([
    [0, 1.5, 3.8],
    [0, 22.5, 3.8],
    [0, 22.5, 18.8],
    [0, 1.5, 18.8],
    [24.3, 0, 3.8],
    [3.3, 0, 3.8],
    [3.3, 0, 18.8],
    [24.3, 0, 18.8]
])

points_2d_left = np.array([(251, 379), (84, 352), (91, 201), (259, 190)])
points_2d_right = np.array([(475, 373), (312, 402), (323, 185), (478, 214)])
points_2d = np.vstack((points_2d_left, points_2d_right))

# Compute 3D range
range_3d = np.max(points_3d, axis=0) - np.min(points_3d, axis=0)

# Compute 2D range
range_2d = np.max(points_2d, axis=0) - np.min(points_2d, axis=0)

# Compute scaling factors
scale_factors = range_2d / range_3d[:2]

# Scale the 3D points
points_3d_scaled = points_3d.copy()
points_3d_scaled[:, 0] *= scale_factors[0]
points_3d_scaled[:, 1] *= scale_factors[1]

print("Original 3D points:")
print(points_3d)
print("\nScaled 3D points:")
print(points_3d_scaled)


Original 3D points:
[[ 0.   1.5  3.8]
 [ 0.  22.5  3.8]
 [ 0.  22.5 18.8]
 [ 0.   1.5 18.8]
 [24.3  0.   3.8]
 [ 3.3  0.   3.8]
 [ 3.3  0.  18.8]
 [24.3  0.  18.8]]

Scaled 3D points:
[[  0.          14.46666667   3.8       ]
 [  0.         217.           3.8       ]
 [  0.         217.          18.8       ]
 [  0.          14.46666667  18.8       ]
 [394.           0.           3.8       ]
 [ 53.50617284   0.           3.8       ]
 [ 53.50617284   0.          18.8       ]
 [394.           0.          18.8       ]]


In [4]:
points_3d_scaled

array([[  0.        ,  14.46666667,   3.8       ],
       [  0.        , 217.        ,   3.8       ],
       [  0.        , 217.        ,  18.8       ],
       [  0.        ,  14.46666667,  18.8       ],
       [394.        ,   0.        ,   3.8       ],
       [ 53.50617284,   0.        ,   3.8       ],
       [ 53.50617284,   0.        ,  18.8       ],
       [394.        ,   0.        ,  18.8       ]])

In [14]:

import numpy as np
import cv2

# Image dimensions
width = 1280
height = 720

# Initialize intrinsic matrix
fx = width
fy = height
cx = width / 2
cy = height / 2
intrinsic_matrix = np.array([[fx, 0, cx],
                             [0, fy, cy],
                             [0, 0, 1]], dtype=np.float32)

# Initial distortion coefficients
dist_coeff = np.zeros((5,1), dtype=np.float32)

# Your 3D and 2D points
scaled_3d_points = np.array([
    [0, 14.46666667, 3.8],
    [0, 217, 3.8],
    [0, 217, 18.8],
    [0, 14.46666667, 18.8],
    [394, 0, 3.8],
    [53.50617284, 0, 3.8],
    [53.50617284, 0, 18.8],
    [394, 0, 18.8]
], dtype=np.float32)

points_2d_left = np.array([(251, 379), (84, 352), (91, 201), (259, 190)], dtype=np.float32)
points_2d_right = np.array([(475, 373), (312, 402), (323, 185), (478, 214)], dtype=np.float32)

# Camera Calibration
flags = 0
flags |= cv2.CALIB_FIX_INTRINSIC
ret, mtx_left, dist_left, rvecs_left, tvecs_left = cv2.calibrateCamera([scaled_3d_points[:4]], [points_2d_left], (width, height), intrinsic_matrix, dist_coeff, flags=flags)
ret, mtx_right, dist_right, rvecs_right, tvecs_right = cv2.calibrateCamera([scaled_3d_points[4:]], [points_2d_right], (width, height), intrinsic_matrix, dist_coeff, flags=flags)

print("Left Camera Matrix:", mtx_left)
print("Right Camera Matrix:", mtx_right)

error: OpenCV(4.7.0) /Users/xperience/GHA-OCV-Python/_work/opencv-python/opencv-python/opencv/modules/calib3d/src/calibration.cpp:1576: error: (-5:Bad argument) For non-planar calibration rigs the initial intrinsic matrix must be specified in function 'cvCalibrateCamera2Internal'


In [22]:
import cv2
import numpy as np

# Reading the Images
left_img = cv2.imread("im1.jpeg")
right_img = cv2.imread("im2.jpeg")

# Known 3D world points and 2D image points
world_points = np.array([
    [0, 1.5, 0],
    [0, 22.5, 0],
    [0, 22.5, 0],
    [0, 1.5, 0],
    [24.3, 0, 0],
    [3.3, 0, 0],
    [3.3, 0, 0],
    [24.3, 0, 0]
], dtype=np.float32)

points_2d_left = np.array([
    [251, 379], 
    [84, 352], 
    [91, 201], 
    [259, 190]
], dtype=np.float32)

points_2d_right = np.array([
    [475, 373],
    [312, 402], 
    [323, 185], 
    [478, 214]
], dtype=np.float32)

# Camera calibration
# Define image size
image_size = (left_img.shape[1], left_img.shape[0])  # (width, height)

# Initial camera matrix - assuming roughly central principal point and aspect ratio = 1
initial_camera_matrix = np.array([[1000, 0, image_size[0]/2], [0, 1000, image_size[1]/2], [0, 0, 1]])
ret, mtx_left, dist_left, rvecs_left, tvecs_left = cv2.calibrateCamera([world_points[:4]], [points_2d_left], image_size, None, None)
ret, mtx_right, dist_right, rvecs_right, tvecs_right = cv2.calibrateCamera([world_points[4:]], [points_2d_right], image_size, None, None)


# Compute the fundamental matrix
F, _ = cv2.findFundamentalMat(points_2d_left, points_2d_right, cv2.FM_LMEDS)

def mouse_event_handler(event, x, y, flags, param):
    global left_img, right_img
    
    if event == cv2.EVENT_LBUTTONDOWN:
        if param == "left":
            cv2.circle(left_img, (x,y), 5, (0,0,255), -1)
            
            # Calculate epipolar line on the right image
            line = cv2.computeCorrespondEpilines(np.array([[x, y]]).reshape(-1,1,2), 1, F)
            line = line.squeeze()
            start = (0, int(-line[2]/line[1]))
            end = (right_img.shape[1], int((-line[2]-line[0]*right_img.shape[1])/line[1]))
            
            cv2.line(right_img, start, end, (0,255,0), 1)
            
            # Triangulation to compute 3D coordinates
            P1 = np.hstack((mtx_left, np.array([[0], [0], [0]])))
            P2 = np.hstack((mtx_right, np.array([[0], [0], [0]])))

            point_4d_hom = cv2.triangulatePoints(P1, P2, np.array([[x], [y]]), np.array([[x], [y]]))  # You may need to adjust the corresponding point in the right image
            point_3d = point_4d_hom / point_4d_hom[3]
            print("3D Point:", point_3d[:3])

        elif param == "right":
            # Similar steps for the right image...
            cv2.circle(right_img, (x,y), 5, (0,0,255), -1)
            
            # Calculate epipolar line on the right image
            line = cv2.computeCorrespondEpilines(np.array([[x, y]]).reshape(-1,1,2), 1, F)
            line = line.squeeze()
            start = (0, int(-line[2]/line[1]))
            end = (right_img.shape[1], int((-line[2]-line[0]*right_img.shape[1])/line[1]))
            
            cv2.line(right_img, start, end, (0,255,0), 1)
            
            # Triangulation to compute 3D coordinates
            P1 = np.hstack((mtx_left, np.array([[0], [0], [0]])))
            P2 = np.hstack((mtx_right, np.array([[0], [0], [0]])))

            point_4d_hom = cv2.triangulatePoints(P1, P2, np.array([[x], [y]]), np.array([[x], [y]]))  # You may need to adjust the corresponding point in the right image
            point_3d = point_4d_hom / point_4d_hom[3]
            print("3D Point:", point_3d[:3])
            pass

        cv2.imshow("Left Image", left_img)
        cv2.imshow("Right Image", right_img)

cv2.namedWindow("Left Image")
cv2.namedWindow("Right Image")

cv2.setMouseCallback("Left Image", mouse_event_handler, "left")
cv2.setMouseCallback("Right Image", mouse_event_handler, "right")

cv2.imshow("Left Image", left_img)
cv2.imshow("Right Image", right_img)
cv2.waitKey(0)
cv2.destroyAllWindows()


error: OpenCV(4.7.0) /Users/xperience/GHA-OCV-Python/_work/opencv-python/opencv-python/opencv/modules/calib3d/src/calibration.cpp:1173: error: (-2:Unspecified error) in function 'void cvFindExtrinsicCameraParams2(const CvMat *, const CvMat *, const CvMat *, const CvMat *, CvMat *, CvMat *, int)'
> DLT algorithm needs at least 6 points for pose estimation from 3D-2D point correspondences. (expected: 'count >= 6'), where
>     'count' is 4
> must be greater than or equal to
>     '6' is 6


In [23]:
import cv2
import numpy as np

# Load the left and right images in grayscale
imgL = cv2.imread('im1.jpeg', 0)
imgR = cv2.imread('im2.jpeg', 0)

# Use StereoSGBM to calculate disparity
window_size = 5
min_disp = 32
num_disp = 112-min_disp
stereo = cv2.StereoSGBM_create(minDisparity = min_disp,
    numDisparities = num_disp,
    blockSize = 16,
    P1 = 8*3*window_size**2,
    P2 = 32*3*window_size**2,
    disp12MaxDiff = 1,
    uniquenessRatio = 10,
    speckleWindowSize = 100,
    speckleRange = 32
)

In [24]:
disparity = stereo.compute(imgL,imgR).astype(np.float32) / 16.0


In [25]:
disparity

array([[31., 31., 31., ..., 31., 31., 31.],
       [31., 31., 31., ..., 31., 31., 31.],
       [31., 31., 31., ..., 31., 31., 31.],
       ...,
       [31., 31., 31., ..., 31., 31., 31.],
       [31., 31., 31., ..., 31., 31., 31.],
       [31., 31., 31., ..., 31., 31., 31.]], dtype=float32)

In [31]:
h, w = imgL.shape[:2]
f = 0.8 * w  # guess for focal length
Q = np.float32([[1, 0, 0, -0.5 * w],
                    [0, -1, 0, 0.5 * h],  # turn points 180 deg around x-axis,
                    [0, 0, 0, -f],  # so that y-axis looks up
                    [0, 0, 1, 0]])

In [32]:
points = cv2.reprojectImageTo3D(disparity, Q)

In [33]:
points

array([[[-10.32258  ,   7.7419353, -16.516129 ],
        [-10.290322 ,   7.7419353, -16.516129 ],
        [-10.258064 ,   7.7419353, -16.516129 ],
        ...,
        [ 10.225806 ,   7.7419353, -16.516129 ],
        [ 10.258064 ,   7.7419353, -16.516129 ],
        [ 10.290322 ,   7.7419353, -16.516129 ]],

       [[-10.32258  ,   7.709677 , -16.516129 ],
        [-10.290322 ,   7.709677 , -16.516129 ],
        [-10.258064 ,   7.709677 , -16.516129 ],
        ...,
        [ 10.225806 ,   7.709677 , -16.516129 ],
        [ 10.258064 ,   7.709677 , -16.516129 ],
        [ 10.290322 ,   7.709677 , -16.516129 ]],

       [[-10.32258  ,   7.677419 , -16.516129 ],
        [-10.290322 ,   7.677419 , -16.516129 ],
        [-10.258064 ,   7.677419 , -16.516129 ],
        ...,
        [ 10.225806 ,   7.677419 , -16.516129 ],
        [ 10.258064 ,   7.677419 , -16.516129 ],
        [ 10.290322 ,   7.677419 , -16.516129 ]],

       ...,

       [[-10.32258  ,  -7.645161 , -16.516129 ],
        [-1

: 

In [30]:
left_img
cv2.findChessboardCorners(left_img, (8,7))

(False, None)

In [29]:
left_img

array([[[103, 103, 103],
        [103, 103, 103],
        [103, 103, 103],
        ...,
        [ 52,  52,  52],
        [ 42,  42,  42],
        [ 62,  62,  62]],

       [[104, 104, 104],
        [103, 103, 103],
        [104, 104, 104],
        ...,
        [ 52,  52,  52],
        [ 42,  42,  42],
        [ 65,  65,  65]],

       [[102, 102, 102],
        [101, 101, 101],
        [104, 104, 104],
        ...,
        [ 52,  52,  52],
        [ 41,  41,  41],
        [ 67,  67,  67]],

       ...,

       [[ 43,  43,  43],
        [ 45,  45,  45],
        [ 46,  46,  46],
        ...,
        [ 29,  29,  29],
        [ 29,  29,  29],
        [ 27,  27,  27]],

       [[ 45,  45,  45],
        [ 45,  45,  45],
        [ 51,  51,  51],
        ...,
        [ 28,  28,  28],
        [ 29,  29,  29],
        [ 27,  27,  27]],

       [[ 46,  46,  46],
        [ 48,  48,  48],
        [ 55,  55,  55],
        ...,
        [ 31,  31,  31],
        [ 30,  30,  30],
        [ 29,  29,  29]]

In [2]:
import cv2
left = cv2.imread('im1.jpeg')

In [14]:
gray = cv2.cvtColor(left, cv2.COLOR_BGR2GRAY)
cv2.findChessboardCornersSBWithMeta(gray, (6,4), None)

(False, None, None)

In [5]:
import numpy as np
import cv2

# Define 3D world coordinates of 4 known points
world_points = np.array([
    [0, 1.5, 3.8],
    [0, 22.5, 3.8],
    [0, 22.5, 18.8],
    [0, 1.5, 18.8],
    [24.3, 0, 3.8],
    [3.3, 0, 3.8],
    [3.3, 0, 18.8],
    [24.3, 0, 18.8]
], dtype=np.float32).reshape(1, -1, 3)

# Define corresponding 2D image points
#407 356 
points_2d_left = np.array([
    (251, 379),
    (84, 352),
    (91, 201),
    (259, 190),
    (407, 356),
    (294, 379),
    (300, 195),
    (411, 219),
], dtype=np.float32).reshape(1, -1, 2)

# Calibrate the camera
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(world_points, points_2d_left, (1280, 720), None, None)

# Print the camera matrix
print("Camera Matrix:")
print(mtx)

error: OpenCV(4.7.0) /Users/xperience/GHA-OCV-Python/_work/opencv-python/opencv-python/opencv/modules/calib3d/src/calibration.cpp:1576: error: (-5:Bad argument) For non-planar calibration rigs the initial intrinsic matrix must be specified in function 'cvCalibrateCamera2Internal'


In [1]:
import math
import cv2
import numpy as np


with open('im1_points.txt', 'r') as f:
    points1 = f.readlines()

with open('im2_points.txt', 'r') as f:
    points2 = f.readlines()

with open('3DCoordinates.txt', 'r') as f:
    points_3d = f.readlines()

points1 = [point.split() for point in points1]  # 2D image points
points2 = [point.split() for point in points2]  # 2D image points
points_3d = [point.split() for point in points_3d]  # 3D world points
im1 = cv2.imread('im1.jpeg', cv2.IMREAD_GRAYSCALE)
im2 = cv2.imread('im2.jpeg', cv2.IMREAD_GRAYSCALE)


def camera_calibration(image_points, world_points):
    A = np.zeros((len(image_points) * 2, 12))
    for i, (image, object) in enumerate(zip(image_points, world_points)):
        X, Y, Z = [float(point) for point in object]
        x, y = [float(point) for point in image]
        A[2 * i, :] = [-X, -Y, -Z, -1, 0, 0, 0, 0, x * X, x * Y, x * Z, x]
        A[2 * i + 1, :] = [0, 0, 0, 0, -X, -Y, -Z, -1, y * X, y * Y, y * Z, y]
    # Perform SVD decomposition
    _, _, V = np.linalg.svd(A)
    P = V[-1].reshape((3, 4))

    # Extract camera parameters from V
    camera_params = V[-1, :12]
    camera_params /= camera_params[-1]  # Normalize the parameters

    print(f'Camera parameters: \n {camera_params}')
    # Compute the calibration error
    projection_error = 0.0
    for i, (image, object) in enumerate(zip(image_points, world_points)):
        X = [float(point) for point in object]
        X.append(1)
        x = [float(point) for point in image]
        x.append(1)

        projected_x = np.dot(camera_params.reshape((3, 4)), X)
        projected_x /= projected_x[-1]  # Normalizing
        projection_error += np.linalg.norm(projected_x[:-1] - x[:-1])

    mean_error = projection_error / len(image_points)
    print("Mean reprojection error: {}".format(mean_error))

    return camera_params


def calculate_fundamental_matrix(img1, img2):
    sift = cv2.SIFT_create()

    # Detect keypoints and compute descriptors for both images
    keypoints1, descriptors1 = sift.detectAndCompute(img1, None)
    keypoints2, descriptors2 = sift.detectAndCompute(img2, None)

    # Initialize Brute-Force Matcher
    bf = cv2.BFMatcher()

    # Match descriptors
    matches = bf.knnMatch(descriptors1, descriptors2, k=2)

    # Apply ratio test to keep good matches
    good_matches = []
    for m, n in matches:
        if m.distance < 0.75 * n.distance:
            good_matches.append(m)

    # Extract corresponding keypoints
    src_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)
    dst_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)

    # Compute fundamental matrix using RANSAC
    fundamental_matrix, mask = cv2.findFundamentalMat(src_pts, dst_pts, cv2.FM_RANSAC)

    print(fundamental_matrix)
    return fundamental_matrix


def calculate_zncc(img1, img2, point1, point2, window_size):
    """
    Calculate Zero Mean Normalized Cross-Correlation (ZNCC) between two pixel points.

    Args:
        img1 (numpy.ndarray): The first image.
        img2 (numpy.ndarray): The second image.
        point1 (tuple): Coordinates (row, col) of the first point in img1.
        point2 (tuple): Coordinates (row, col) of the corresponding point in img2.
        window_size (int): Size of the window around the points for ZNCC calculation.

    Returns:
        zncc_score (float): ZNCC score between the patches centered at the two points.
    """
    half_window = window_size // 2

    # Extract patches around the points
    patch1 = img1[point1[0] - half_window:point1[0] + half_window + 1,
             point1[1] - half_window:point1[1] + half_window + 1]

    patch2 = img2[point2[0] - half_window:point2[0] + half_window + 1,
             point2[1] - half_window:point2[1] + half_window + 1]

    # Calculate means of the patches
    mean1 = np.mean(patch1)
    mean2 = np.mean(patch2)

    # Calculate zero-mean patches
    zero_mean_patch1 = patch1 - mean1
    zero_mean_patch2 = patch2 - mean2

    # Calculate the cross-correlation
    cross_corr = np.sum(zero_mean_patch1 * zero_mean_patch2)

    # Calculate the standard deviations
    std_dev1 = np.sqrt(np.sum(zero_mean_patch1 ** 2))
    std_dev2 = np.sqrt(np.sum(zero_mean_patch2 ** 2))

    # Calculate ZNCC
    zncc_score = cross_corr / (std_dev1 * std_dev2)

    return zncc_score


def drawEpipolarLine(u, v, F, img1, img2):

    p = np.array([[u, v, 1]], dtype=np.float32)

    sift = cv2.SIFT_create()

    # Detect keypoints
    keypoints1, desc1 = sift.detectAndCompute(img1, None)
    keypoints2, desc2 = sift.detectAndCompute(img2, None)

    # Create a Brute-Force Matcher
    bf = cv2.BFMatcher()
    # Match descriptors from both images
    matches = bf.knnMatch(desc1, desc2, k=2)
    # Apply ratio test to filter good matches
    good_matches = []
    for m, n in matches:
        if m.distance < 0.3 * n.distance:
            good_matches.append(m)

    src_pts = np.float32([keypoints1[match.queryIdx].pt for match in good_matches])
    dst_pts = np.float32([keypoints2[match.trainIdx].pt for match in good_matches])

    distance_from_keypoints = list()
    for i, pt in enumerate(src_pts):
        distance_from_keypoints.append((math.sqrt((pt[0] - u)**2 + (pt[1] - v)**2), pt, dst_pts[i]))
    distance_from_keypoints.sort(key=lambda x: x[0])
    distance_from_keypoints = distance_from_keypoints[0]

    disparity = distance_from_keypoints[1][0] - distance_from_keypoints[2][0]

    line = cv2.computeCorrespondEpilines(p, 1, F)
    epipolar_line = line.reshape(-1, 3)
    # Calculate epipolar line parameters
    epipolar_line = epipolar_line.flatten()
    a, b, c = epipolar_line

    # Calculate the range of x coordinates for drawing the line
    img1_height, img1_width = img1.shape[:2]
    x1 = 0
    y1 = int(-c / b)
    x2 = img1_width - 1
    y2 = int(-(a * x2 + c) / b)

    img_with_line = cv2.line(img2, [x1, y1], [x2, y2], (0, 255, 0))

    points_right = list()
    ranges = [int(u-disparity), int(u+disparity)]
    for x in range(min(ranges), max(ranges)):
        y = int(-(epipolar_line[0] * x + epipolar_line[2]) / epipolar_line[1])

        try:
            zncc_score = calculate_zncc(img1, img2, (u, v), (x, y), window_size=7)
            points_right.append(((x, y), zncc_score))
        except ValueError:
            continue

    points_right.sort(key=lambda x: x[1], reverse=True)
    best_point_right = points_right[0]

    cv2.circle(img_with_line, (best_point_right[0][0], best_point_right[0][1]), 5, (0, 0, 255), -1)
    cv2.imshow('Image', img_with_line)
    cv2.waitKey(0)
    print(f'the best point {best_point_right[0]}')
    return best_point_right[0]


def three_dimensional_reconstruction(camera_matrix1, camera_matrix2, p1, p2):
    A = np.zeros((4, 3))

    A[0][0] = p1[0] * camera_matrix1[8] - camera_matrix1[0]
    A[0][1] = p1[0] * camera_matrix1[9] - camera_matrix1[1]
    A[0][2] = p1[0] * camera_matrix1[10] - camera_matrix1[2]

    A[1][0] = p1[1] * camera_matrix1[8] - camera_matrix1[4]
    A[1][1] = p1[1] * camera_matrix1[9] - camera_matrix1[5]
    A[1][2] = p1[1] * camera_matrix1[10] - camera_matrix1[6]

    A[2][0] = p2[0] * camera_matrix2[8] - camera_matrix2[1]
    A[2][1] = p2[0] * camera_matrix2[9] - camera_matrix2[2]
    A[2][2] = p2[0] * camera_matrix2[10] - camera_matrix2[3]

    A[3][0] = p2[0] * camera_matrix2[8] - camera_matrix2[4]
    A[3][1] = p2[0] * camera_matrix2[9] - camera_matrix2[5]
    A[3][2] = p2[0] * camera_matrix2[10] - camera_matrix2[6]

    d = np.zeros((1, 4))
    d[0][0] = p1[0] * camera_matrix1[11] - camera_matrix1[3]
    d[0][1] = p1[1] * camera_matrix1[11] - camera_matrix1[7]
    d[0][2] = p2[0] * camera_matrix2[11] - camera_matrix2[3]
    d[0][3] = p2[1] * camera_matrix2[11] - camera_matrix2[7]

    U, S, Vt = np.linalg.svd(A)
    # Calculate the pseudoinverse of S
    S_pseudo = np.zeros(A.shape).T
    S_pseudo[:S.shape[0], :S.shape[0]] = np.diag(1 / S)
    # Calculate the pseudoinverse of A using SVD
    A_pseudo = np.dot(np.dot(Vt.T, S_pseudo), U.T)

    # Solve for matrix X
    X = np.dot(A_pseudo, d.T)

    return X

M1 = camera_calibration(points1, points_3d)
M2 = camera_calibration(points2, points_3d)
F = calculate_fundamental_matrix(im1, im2)

def click_and_mark(event, x, y, flags, param):
    global points
    if event == cv2.EVENT_LBUTTONDOWN:
        #points = (x, y)
        cv2.circle(im1, (x, y), 5, (0, 0, 255), -1)
        cv2.imshow('Image1', im1)# Mark the clicked point with a red circle

        p_ = drawEpipolarLine(x, y, F, im1, im2)
        print('3D Reconstruction: {}'.format(three_dimensional_reconstruction(M1, M2, (x, y), p_)))


cv2.imshow('Image1', im1)
cv2.setMouseCallback('Image1', click_and_mark)

while True:
    key = cv2.waitKey(1) & 0xFF
    # Press 'q' to quit and save the points
    if key == ord('q'):
        break

Camera parameters: 
 [ 1.28673584e+01 -7.08022743e+00  7.63836467e-01  2.65973054e+02
  5.08216239e+00  3.26684771e+00 -1.28100314e+01  4.33572277e+02
  1.74581898e-02  1.32772732e-02  1.01479734e-03  1.00000000e+00]
Mean reprojection error: 0.4362484913196166
Camera parameters: 
 [ 1.68842575e+01 -5.62662763e+00  9.04067537e-01  2.71937129e+02
  5.45493674e+00  4.64392716e+00 -1.54990563e+01  4.69532176e+02
  1.82295906e-02  1.83503713e-02  7.30327158e-04  1.00000000e+00]
Mean reprojection error: 0.7810626650427404
[[ 1.81027797e-07  4.21788655e-05 -1.18716237e-02]
 [-4.96572594e-05 -7.02350956e-07  4.55563961e-02]
 [ 1.39292079e-02 -4.94497562e-02  1.00000000e+00]]
the best point (254, 403)
3D Reconstruction: [[-13.85506002]
 [-12.78419918]
 [ -0.64817932]]
the best point (85, 364)
3D Reconstruction: [[17.06386603]
 [ 1.54519934]
 [ 1.11012526]]
the best point (98, 196)
3D Reconstruction: [[54.47631393]
 [52.48770536]
 [ 2.28063004]]


KeyboardInterrupt: 

In [6]:
points_2d_left
img1 = cv2.imread('im1.jpeg')
im2 = cv2.imread('im2.jpeg')

ret, M1, dist, rvecs, tvecs = cv2.calibrateCamera(world_points, points_2d_left, img1.shape[::-1], None, None)
ret, M2, dist, rvecs, tvecs = cv2.calibrateCamera(world_points, points_2d_right, img2.shape[::-1], None, None)

NameError: name 'img1' is not defined